# 0. Overview

# 1. Import Libraries and Constants

In [2]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
RAND_STATE = 42

# 2. Data Cleaning

In [3]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, pre_trained_embeddings=None):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2) # Stacked LSTMs
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Adjust for bidirectional LSTM
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        out = self.relu(out)
        q_values = self.fc(out)
        return q_values

In [13]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [8]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[71,  8, 66, 53, 28,  0,  0,  0,  0,  0],
        [76, 57, 13, 38,  0,  0,  0,  0,  0,  0],
        [31, 66,  6, 36, 20, 32,  0,  0,  0,  0],
        [16, 44, 66, 14, 29,  0,  0,  0,  0,  0]])
Train Label: tensor([1, 1, 0, 1])
Validation Encoded text: tensor([[ 5, 24,  1,  2,  0,  0,  0,  0,  0,  0],
        [30, 18,  0,  0,  0,  0,  0,  0,  0,  0],
        [31, 16, 11, 22,  0,  0,  0,  0,  0,  0],
        [29, 27, 28, 17,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[23, 20, 22, 29,  0,  0,  0,  0,  0,  0],
        [14, 12, 28,  1,  0,  0,  0,  0,  0,  0],
        [ 4, 33, 25, 27, 11,  0,  0,  0,  0,  0],
        [30, 24,  2, 13, 30, 21, 18,  0,  0,  0]])
Test Label: tensor([1, 0, 0, 0])


In [14]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.686912

Validation set: Average loss: 0.1733, Accuracy: 4/8 (50%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.648722

Validation set: Average loss: 0.1736, Accuracy: 4/8 (50%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.667809

Validation set: Average loss: 0.1732, Accuracy: 4/8 (50%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.671954

Validation set: Average loss: 0.1731, Accuracy: 4/8 (50%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.692263

Validation set: Average loss: 0.1730, Accuracy: 4/8 (50%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.697424

Validation set: Average loss: 0.1730, Accuracy: 6/8 (75%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.669491

Validation set: Average loss: 0.1731, Accuracy: 4/8 (50%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.709306

Validation set: Average loss: 0.1725, Accuracy: 5/8 (62%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.668036

Validation set: Average loss: 0.1721, Accuracy: 4/8 (50%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.719614

Validation set: Average loss: 0.1711, 

In [ ]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 30
learning_rate = 0.001

def objective(trial):
    # Define the search space
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
    
    # Objective: maximize validation accuracy
    print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    return val_loss, val_accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'Value: {trial.value}')
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [18]:
optuna.study

<module 'optuna.study' from 'c:\\Users\\supha\\repos\\cs5756\\RL-Improve-Promotion-Section\\.venv\\Lib\\site-packages\\optuna\\study\\__init__.py'>